# SQLMesh Runner

## Set env vars

In [ ]:
import os

keyvault = "mattiasthalen-fabric"

env_vars = [
    "CREDENTIALS__AZURE_TENANT_ID",
    "CREDENTIALS__AZURE_CLIENT_ID",
    "CREDENTIALS__AZURE_CLIENT_SECRET",
    "FABRIC__WORKSPACE_ID",
    "FABRIC__WAREHOUSE_ENDPOINT",
    "FABRIC__STATE_ENDPOINT",
    "FABRIC__STATE_DATABASE"
]

for env_var in env_vars:
    secret = env_var.replace("_", "-")
    value = notebookutils.credentials.getSecret(f"https://{keyvault}.vault.azure.net/", secret)
    os.environ[env_var] = value

    print(f"{env_var} = {os.getenv(env_var)}")

## Clone SQLMesh project

In [ ]:
import git

organization = "mattiasthalen"
repo_name = "sqlmesh-fabric-demo"

url = f"https://github.com/{organization}/{repo_name}.git"

branch = "main"
code_path = f"/lakehouse/default/Files/{repo_name}"

try:
    repo = git.Repo.clone_from(
        url=url,
        to_path=code_path,
        branch=branch,
        depth=1,
        single_branch=True
    )

    print(f"Cloned repo to {code_path}")

except git.GitCommandError as e:
    if e.status == 128:
        repo = git.Repo(code_path)
        origin = repo.remotes.origin
        repo.git.reset('--hard', f'origin/{branch}')
        origin.pull(branch)

        print("Pulled latest changes from repo")
        
    else:
        raise

## Run SQLMesh

In [ ]:
%pip install "sqlmesh[fabric,mssql] @ git+https://github.com/TobikoData/sqlmesh.git"

In [ ]:
import subprocess

cmd = ["sqlmesh", "plan", "prod", "--run", "--auto-apply", "--no-prompts"]
result = subprocess.run(
    cmd,
    cwd=code_path,
    text=True,
    check=True
) 